<a href="https://colab.research.google.com/github/ankit-genzeon/AI-ML-Bootcamp-Genzeon-2023/blob/master/C10_twitter_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
dataset = '/content/drive/MyDrive/Colab Notebooks/DataSet/Tweets.csv'
df = pd.read_csv(dataset)
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [36]:
df.drop(['textID','text'], axis=1, inplace =True)
print (df.head())

                         selected_text sentiment
0  I`d have responded, if I were going   neutral
1                             Sooo SAD  negative
2                          bullying me  negative
3                       leave me alone  negative
4                        Sons of ****,  negative


In [37]:
# Fill missing values in 'selected_text' column with empty strings
df['selected_text'].fillna('', inplace=True)

In [38]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'sentiment' column in your DataFrame
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'])

# Print the mapping of original sentiment labels to encoded values
print("Sentiment Labels:")
for sentiment, encoded_value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(sentiment, ":", encoded_value)


Sentiment Labels:
negative : 0
neutral : 1
positive : 2


In [39]:
df.head()

,selected_text,sentiment,sentiment_encoded
0,"I`d have responded, if I were going",neutral,1
1,Sooo SAD,negative,0
2,bullying me,negative,0
3,leave me alone,negative,0
4,"Sons of ****,",negative,0


In [40]:
import nltk
nltk.download('wordnet')
nltk.download ('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def tokenizer(text):
    tokens = text.lower().split()
    tagged_tokens = pos_tag(tokens)

    lemmatized_tokens = []
    for token, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        if pos:
            lemmatized_token = lemmatizer.lemmatize(token, pos=pos)
        else:
            lemmatized_token = lemmatizer.lemmatize(token)
        lemmatized_tokens.append(lemmatized_token)

    return lemmatized_tokens

# Helper function to convert POS tag to WordNet POS tag
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [42]:
#split
X = df['selected_text']
y= df['sentiment_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [44]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Step 4: Train the model
model = SVC()
model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.70      0.77      1582
           1       0.74      0.92      0.82      2250
           2       0.91      0.78      0.84      1665

    accuracy                           0.81      5497
   macro avg       0.84      0.80      0.81      5497
weighted avg       0.83      0.81      0.81      5497



In [45]:
# Step 5: Evaluate the model
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy*100,'%')

Accuracy: 81.40804074949972 %


In [46]:
# Step 6: Use the trained model for prediction on new data
new_comments = ["This is a friendly conversation.", "You're such a loser!"]
new_comments_transformed = vectorizer.transform(new_comments)
predictions = model.predict(new_comments_transformed)
print(predictions)

[2 0]


In [ ]:
pip install gradio

In [48]:
import gradio as gr



# Define the function to make predictions
def predict_sentiment(comment):
    transformed_comment = vectorizer.transform([comment])
    prediction = model.predict(transformed_comment)[0]
    return "Negative" if prediction == 0 else "Positive" if prediction == 2 else "Neutral"

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.inputs.Textbox(placeholder="Enter a comment"),
    outputs="text"
)

# Launch the interface
iface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>